In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
train = pd.read_csv('https://dphi.s3.ap-south-1.amazonaws.com/dataset/train_age_dataset.csv')
test = pd.read_csv('https://dphi.s3.ap-south-1.amazonaws.com/dataset/test_age_dataset.csv')
train.head()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0,0.000000,0.200000,0.0,0.041667,0.025000,0.0,0.000000,0.175000,0.033333,0.0,1
1,405231,51100441,2,2,0.0,0.0,0.0,0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0,0.008475,0.093220,0.0,0.012712,0.018644,0.0,0.084746,0.000000,0.033898,82.5,2
2,57867,6887426,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.341657,22.000000,83.051415,0,0.000000,0.002786,0.0,0.000000,0.000557,0.0,0.002786,0.000000,0.000000,0.0,1
3,272618,50742404,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.005735,3.000000,523.080000,0,0.000000,0.008403,0.0,0.000000,0.001681,0.0,0.000000,0.000000,0.008403,0.0,1
4,251123,45589200,2,2,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.456549,12.333333,53.766640,0,0.000000,0.204918,0.0,0.000000,0.049180,0.0,0.008197,0.057377,0.180328,0.0,1


In [ ]:
train.shape

(488877, 27)

In [ ]:
num_cols = [col for col in X.columns if X[col].nunique()>5]

In [ ]:
# using boxplot to check for outliers and skewedness
plt.figure(figsize=(15,20))
ax = sns.boxplot(data=X[num_cols])
plt.xticks(rotation=90)
plt.show()

In [ ]:
X[num_cols].hist(figsize=(14,10),xlabelsize=8, ylabelsize=8,bins=170)

In [ ]:
for col in num_cols:
    fig,ax = plt.subplots(1,3)
    sns.distplot(X[col],ax=ax[0])
    sns.distplot(np.sqrt(X[col]-X[col].min()+1),ax=ax[1])
    sns.distplot((X[col]-X[col].min()+1).transform(np.log),ax=ax[2])

In [ ]:
train.drop(['Unnamed: 0','userId'],axis=1).describe()

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
count,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,4.888770e+05,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000,488877.000000
mean,1.975382,1.213037,0.082233,0.347520,0.403049,0.739748,0.000277,0.000981,0.012805,0.179148,0.341582,1.087395e+02,83.104654,0.320923,0.017065,0.391013,0.002009,0.074353,0.066927,0.032855,0.141705,0.150401,0.158382,164.765782,1.741643
std,0.430707,0.409454,2.467781,0.816334,0.925291,3.075954,0.012221,0.020381,0.159553,0.647588,0.195441,5.537516e+04,76.779586,7.649827,0.180402,0.905303,0.030561,0.211094,0.174791,0.193962,0.402612,0.451999,0.502065,981.988886,1.088559
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000653,1.000000e+00,0.233000,0.000000,0.000000,0.000887,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199675,5.655172e+00,30.724000,0.000000,0.000000,0.040650,0.000000,0.000000,0.002265,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329743,8.500000e+00,62.501164,0.000000,0.000000,0.124031,0.000000,0.003968,0.015873,0.000000,0.011834,0.014563,0.015873,0.000000,1.000000
75%,2.000000,1.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150183,0.460492,1.308654e+01,112.246500,0.000000,0.009091,0.364486,0.000000,0.060000,0.059016,0.000000,0.117188,0.118644,0.122396,178.666667,2.000000
max,3.000000,2.000000,895.304000,4.000000,4.000000,624.000000,2.333333,3.000000,27.333333,262.666667,1.000000,3.826604e+07,7541.026000,3228.000000,63.388889,75.662281,8.196850,17.201754,18.756140,19.612903,27.905983,45.083333,55.153846,39304.000000,4.000000


In [ ]:
max_threshold=train.avgTimeSpent.quantile(0.999)
min_threshold=train.avgTimeSpent.quantile(0.001)
print(min_threshold,max_threshold)

1.0 1127.7528305402795


In [ ]:
train[train.avgTimeSpent>max_threshold]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
3430,360998,70780891,2,2,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.694224,4375.500000,56.807167,0,0.000000,0.107143,0.0,0.026786,0.010714,0.0,0.000000,0.089286,0.017857,0.000000,1
6567,237098,42302751,2,1,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.542002,2805.750000,32.251667,0,0.000000,0.097561,0.0,0.000000,0.019512,0.0,0.081301,0.000000,0.016260,0.000000,1
7632,90655,2411472,2,1,0.005906,0.0,0.000000,0,0.0,0.0,0.0,0.294118,0.626412,2170.666667,158.757231,0,0.005906,0.025591,0.0,0.000000,0.010630,0.0,0.000000,0.045276,0.007874,191.071429,1
8072,126380,37543286,2,1,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.100000,0.290098,1204.714286,209.482286,0,0.016000,0.056000,0.0,0.024000,0.001600,0.0,0.008000,0.040000,0.008000,170.000000,4
9993,352172,68511479,2,1,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.445461,4648.800000,50.620800,0,0.000000,0.161290,0.0,0.072581,0.003226,0.0,0.080645,0.032258,0.048387,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482259,31866,45887836,1,1,0.000000,0.0,0.000000,2,0.0,0.0,0.0,0.333333,0.355198,6149.047619,96.447647,0,0.016393,0.147541,0.0,0.012295,0.029508,0.0,0.000000,0.040984,0.131148,169.666667,3
482313,428223,71931471,2,2,0.000000,0.0,0.000000,0,0.0,0.0,0.0,1.166667,0.676014,2195.550000,86.583000,0,0.019608,0.078431,0.0,0.000000,0.078431,0.0,0.352941,0.039216,0.000000,154.000000,4
483436,296361,55570615,2,1,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.450884,6735.800000,13.794500,0,0.000000,0.045455,0.0,0.009091,0.005455,0.0,0.018182,0.009091,0.018182,0.000000,1
485760,239576,42852502,2,1,0.000000,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.664751,1220.500000,51.884250,0,0.000000,0.032520,0.0,0.000000,0.006504,0.0,0.000000,0.032520,0.000000,0.000000,1


In [ ]:
train[train.avgTimeSpent<min_threshold]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group


In [ ]:
train1=train[(train.avgTimeSpent<max_threshold)]
train1.shape

(488388, 27)

In [ ]:
min_threshold2,max_threshold2=train.avgt2.quantile([0.001,0.999])
min_threshold2,max_threshold2

(0.0, 17306.76794472008)

In [ ]:
train[train.avgt2>max_threshold2]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
3912,162104,568675,3,1,0.004127,3.0,3.0,0,0.0,0.0,0.0,0.115152,0.741778,11.399083,18.281810,1,0.004127,0.220083,0.001376,0.004127,0.058322,0.000000,0.008253,0.291609,0.000000,20476.44444,4
5673,166895,434059,3,1,0.002415,0.0,0.0,0,0.0,0.0,0.0,0.233333,0.587187,151.555556,204.195500,0,0.003623,0.067633,0.000000,0.013889,0.011836,0.016908,0.021739,0.016908,0.031401,23947.75000,4
5700,30356,248129,1,1,0.003205,0.0,0.0,0,0.0,0.0,0.0,0.466667,0.392069,43.400000,86.108333,0,0.003205,0.011752,0.000000,0.001068,0.000641,0.002137,0.000000,0.002137,0.001068,28478.66667,4
5850,432020,444656,2,1,0.004884,0.0,0.0,0,0.0,0.0,0.0,1.068966,0.409658,43.500000,165.177889,2,0.003663,0.028083,0.002442,0.006105,0.002442,0.006105,0.008547,0.007326,0.002442,19914.80000,2
6378,490666,677409,2,1,0.002861,4.0,3.0,0,0.0,0.0,0.0,1.857143,0.497580,13.000000,24.032000,0,0.004292,0.004292,0.000000,0.000000,0.000858,0.000000,0.000000,0.004292,0.000000,18642.66667,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481599,164034,1627169,3,1,0.001776,0.0,0.0,0,0.0,0.0,0.0,0.100000,0.223652,21.375000,146.712571,0,0.001776,0.014210,0.000000,0.000000,0.002842,0.000000,0.014210,0.000000,0.000000,36433.00000,1
485292,465823,667911,2,1,0.001425,0.0,0.0,0,0.0,0.0,0.0,1.571429,0.122018,13.000000,266.227750,0,0.002849,0.005698,0.000000,0.000712,0.000855,0.000000,0.000000,0.005698,0.000000,20069.71429,4
487128,465821,666004,2,1,0.025641,0.0,0.0,0,0.0,0.0,0.0,0.529412,0.418315,26.476190,123.887333,0,0.002849,0.019943,0.000000,0.000000,0.005983,0.011396,0.009972,0.000000,0.008547,21167.28571,4
487605,454572,304110,2,1,0.011013,0.0,0.0,0,0.0,0.0,0.0,1.333333,0.128582,26.200000,247.161500,0,0.003304,0.005507,0.000000,0.002753,0.000000,0.000000,0.004405,0.000000,0.001101,21979.75000,2


In [ ]:
train[train.avgt2<min_threshold2]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group


In [ ]:
train2=train1[(train1.avgt2<max_threshold2)]
train2.shape

(487904, 27)

In [ ]:
min_threshold3,max_threshold3=train.following_rate.quantile([0.001,0.999])
min_threshold3,max_threshold3

(0.0, 9.185326629360251)

In [ ]:
train[train.following_rate>max_threshold3]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
16,16863,32554791,1,1,19.640625,1.921013,1.981818,5,0.0,0.000000,0.000000,0.044444,0.527914,13.243902,56.036605,1,0.109375,2.335938,0.007812,0.296875,0.521875,0.015625,0.929688,1.085938,1.171875,119.328358,3
576,35463,49483564,1,1,106.741667,2.182099,2.134377,0,0.0,0.000000,0.000000,0.242798,0.221895,21.032922,121.378534,2,0.083333,1.916667,0.016667,0.687500,0.535000,0.225000,1.083333,0.991667,1.750000,75.824464,4
1419,508102,39810026,2,2,14.161290,2.037838,2.046584,5,0.0,0.000000,0.000000,0.760000,0.396417,9.840517,65.223093,1,0.201613,0.991935,0.008065,0.588710,0.138710,0.000000,0.750000,0.500000,0.620968,151.163009,2
2089,105452,38187200,2,1,18.376000,2.139896,2.142629,6,0.0,0.000000,0.000000,0.777778,0.389004,6.858491,21.689189,1,0.072000,0.344000,0.008000,0.192000,0.092800,0.256000,0.024000,0.456000,0.112000,133.204852,2
2185,45280,33701418,1,1,23.755906,1.964344,2.109988,4,0.0,0.000000,0.005906,0.096457,0.197285,6.625000,110.349773,23,0.291339,3.519685,0.181102,0.574803,1.155906,0.031496,1.889764,2.826772,2.181102,138.453280,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486601,508005,34467156,2,1,111.181102,1.958876,2.326954,2,0.0,0.000000,0.000000,0.302158,0.422058,7.510799,41.301413,5,0.181102,3.055118,0.039370,1.185039,0.984252,1.086614,1.708661,1.614173,2.881890,124.526551,3
486953,51807,12630810,1,1,28.072874,2.051962,2.657407,3,0.0,0.000000,0.003521,2.558099,0.536991,90.795789,182.283158,20,1.546559,0.668016,0.080972,0.329960,0.252632,0.663968,0.206478,0.327935,0.724696,175.137572,4
487228,121862,51562639,2,1,90.627119,2.070149,2.377087,26,0.0,0.006173,0.000000,0.342593,0.356966,27.143066,56.074552,5,0.762712,1.932203,0.042373,1.059322,0.737288,0.279661,2.737288,2.177966,0.610169,170.155521,2
487333,38817,36917065,1,2,16.808000,2.102941,2.277946,4,0.0,0.000000,0.007692,0.476923,0.434647,12.964000,35.309948,8,0.160000,1.848000,0.064000,0.740000,0.504000,1.960000,0.456000,0.272000,1.312000,170.067847,3


In [ ]:
train3=train2[(train2.following_rate<max_threshold3)]
train3.shape

(487415, 27)

In [ ]:
min_threshold4,max_threshold4=train.max_repetitive_punc.quantile([0.001,0.999])
min_threshold4,max_threshold4

(0.0, 36.0)

In [ ]:
train[train.max_repetitive_punc>max_threshold4]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
586,400987,32866348,2,1,0.007874,3.000000,3.000000,47,0.000000,0.0,0.000000,0.428571,0.469109,13.400000,68.220571,3,0.015748,0.118110,0.023622,0.149606,0.011024,0.062992,0.015748,0.039370,0.236220,138.000000,3
800,537924,37286724,2,1,0.456000,1.562500,2.404762,37,0.000000,0.0,0.000000,0.200000,0.259731,7.750000,85.211364,0,0.016000,0.096000,0.000000,0.036000,0.011200,0.000000,0.088000,0.040000,0.000000,137.800000,1
930,445132,44949026,2,1,0.000000,0.000000,0.000000,47,0.000000,0.0,0.000000,0.000000,0.291753,9.640000,89.300562,0,0.016393,0.147541,0.000000,0.000000,0.040984,0.000000,0.049180,0.155738,0.000000,154.600000,4
6098,415042,1308784,2,1,0.018364,2.250000,2.500000,72,0.000000,0.0,0.000000,0.421053,0.180830,30.850000,157.146900,1,0.005008,0.023372,0.001669,0.000000,0.006678,0.000000,0.001669,0.003339,0.028381,226.400000,4
6797,464990,54213285,2,1,0.000000,0.000000,0.000000,40,0.000000,0.0,0.000000,0.000000,0.722760,15.000000,40.538000,0,0.008772,0.052632,0.000000,0.013158,0.007018,0.000000,0.000000,0.061404,0.000000,175.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482286,153464,2353713,3,1,0.003891,0.000000,0.000000,40,0.000000,0.0,0.000000,0.217391,0.141151,10.857143,190.392111,0,0.005837,0.036965,0.000000,0.000000,0.016342,0.000000,0.029183,0.048638,0.003891,188.266667,1
483600,43160,33653713,1,1,0.023622,1.727273,4.000000,45,0.000000,0.0,0.000000,2.644444,0.311037,6.538117,70.470115,0,0.236220,0.464567,0.000000,0.350394,0.211024,0.000000,0.440945,0.850394,0.464567,136.010811,4
485318,114794,42599444,2,2,0.000000,0.000000,0.000000,46,0.238095,0.0,1.761905,0.333333,0.418093,5.814815,42.921077,0,0.016260,0.146341,0.000000,0.000000,0.043902,0.097561,0.032520,0.065041,0.024390,140.400000,1
485331,119936,39552379,2,1,1.346774,2.187500,2.042553,44,0.000000,0.0,0.000000,2.394161,0.381120,17.511312,88.970693,9,0.419355,1.709677,0.072581,0.592742,0.475806,0.395161,1.725806,0.717742,0.725806,220.414729,2


In [ ]:
train4=train3[(train3.max_repetitive_punc<max_threshold4)]
train4.shape

(486927, 27)

In [ ]:
min_threshold5,max_threshold5=train.userId.quantile([0.001,0.999])
print(min_threshold5,max_threshold5)

91481.05600000001 78884224.1400001


In [ ]:
train[train.userId>max_threshold5]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
644,387752,78925018,2,2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.108043,9.500000,148.138571,0,0.0,0.266667,0.0,0.133333,0.013333,0.000000,0.000000,0.033333,0.300000,0.0,1
2842,387887,78953699,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.193167,3.000000,94.268000,0,0.0,0.068966,0.0,0.000000,0.013793,0.068966,0.000000,0.000000,0.000000,0.0,1
3478,387717,78904348,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.179028,4.000000,70.420000,0,0.0,0.066667,0.0,0.000000,0.013333,0.000000,0.000000,0.000000,0.066667,0.0,1
7399,388104,78997611,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.227719,3.833333,89.144000,0,0.0,0.379310,0.0,0.000000,0.082759,0.000000,0.413793,0.000000,0.000000,0.0,1
8292,388125,79003481,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.163364,3.000000,64.801333,0,0.0,0.103448,0.0,0.000000,0.020690,0.000000,0.103448,0.000000,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483950,388078,78991224,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.192137,3.000000,143.884500,0,0.0,0.068966,0.0,0.000000,0.013793,0.068966,0.000000,0.000000,0.000000,0.0,1
486375,388166,79016279,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.020379,3.000000,183.622000,0,0.0,0.103448,0.0,0.000000,0.020690,0.000000,0.103448,0.000000,0.000000,0.0,1
487429,388201,79028827,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.192919,3.000000,102.640500,0,0.0,0.068966,0.0,0.000000,0.013793,0.000000,0.068966,0.000000,0.000000,0.0,1
488237,387741,78919138,2,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.146806,3.666667,80.184000,0,0.0,0.066667,0.0,0.000000,0.020000,0.000000,0.000000,0.000000,0.100000,0.0,1


In [ ]:
train[train.userId<min_threshold5]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
621,51954,29683,1,1,0.007130,0.000000,0.000000,0,0.0,0.000000,0.000000,0.631579,0.306866,24.769231,165.554500,0,0.002674,0.013369,0.000000,0.002228,0.001426,0.000000,0.007130,0.000000,0.004456,0.000000,2
690,498254,38250,2,2,0.018298,2.562500,2.500000,5,0.0,0.000000,0.000000,0.063903,0.619956,63.387097,138.898632,0,0.010064,0.518756,0.000000,0.003660,0.004209,0.000000,0.004575,0.000915,0.022873,2249.938144,2
6343,17499,29490,1,1,0.008014,0.000000,0.000000,2,0.0,0.000000,0.040404,0.454545,0.331017,15.123288,155.058037,3,0.012467,0.072128,0.002671,0.003117,0.011754,0.000000,0.044524,0.003562,0.016919,384.448087,2
7334,119637,29514,2,1,0.003562,2.370370,4.000000,3,0.0,0.020833,0.104167,1.562500,0.163409,7.787234,140.219533,0,0.013357,0.028495,0.000000,0.028940,0.021906,0.016919,0.089938,0.027605,0.032947,384.775656,2
8640,36918,25332,1,1,0.005286,2.000000,2.000000,0,0.0,0.000000,0.000000,1.000000,0.355462,10.200000,305.004333,0,0.002643,0.007930,0.000000,0.000881,0.000529,0.000000,0.000000,0.002643,0.001762,2.400000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483424,522913,36305,2,1,0.001821,0.000000,0.000000,0,0.0,0.000000,0.000000,1.200000,0.136461,6.200000,143.110500,0,0.002732,0.005464,0.000000,0.000911,0.000546,0.000000,0.000000,0.002732,0.001821,3537.918919,2
486991,145840,88979,3,1,0.053743,3.000000,3.333333,2,0.0,0.000000,0.100000,2.100000,0.341657,8.600000,83.051415,7,0.007678,0.013436,0.006718,0.000480,0.001727,0.000000,0.000000,0.009597,0.000000,389.769231,2
487280,428367,38815,2,1,0.006410,1.500000,1.000000,3,0.0,0.000000,0.000000,0.193548,0.176521,7.894118,148.583825,0,0.004579,0.051282,0.000000,0.005037,0.013553,0.024725,0.001832,0.047619,0.003663,157.071429,2
487736,105750,74545,2,1,0.005736,0.000000,0.000000,0,0.0,0.000000,0.000000,0.234043,0.530261,17.093750,76.225133,0,0.002868,0.041109,0.000000,0.003824,0.004589,0.005736,0.003824,0.012428,0.008604,1.000000,2


In [ ]:
train4[train4.userId<min_threshold5]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
621,51954,29683,1,1,0.007130,0.000000,0.000000,0,0.0,0.000000,0.000000,0.631579,0.306866,24.769231,165.554500,0,0.002674,0.013369,0.000000,0.002228,0.001426,0.000000,0.007130,0.000000,0.004456,0.000000,2
690,498254,38250,2,2,0.018298,2.562500,2.500000,5,0.0,0.000000,0.000000,0.063903,0.619956,63.387097,138.898632,0,0.010064,0.518756,0.000000,0.003660,0.004209,0.000000,0.004575,0.000915,0.022873,2249.938144,2
6343,17499,29490,1,1,0.008014,0.000000,0.000000,2,0.0,0.000000,0.040404,0.454545,0.331017,15.123288,155.058037,3,0.012467,0.072128,0.002671,0.003117,0.011754,0.000000,0.044524,0.003562,0.016919,384.448087,2
7334,119637,29514,2,1,0.003562,2.370370,4.000000,3,0.0,0.020833,0.104167,1.562500,0.163409,7.787234,140.219533,0,0.013357,0.028495,0.000000,0.028940,0.021906,0.016919,0.089938,0.027605,0.032947,384.775656,2
8640,36918,25332,1,1,0.005286,2.000000,2.000000,0,0.0,0.000000,0.000000,1.000000,0.355462,10.200000,305.004333,0,0.002643,0.007930,0.000000,0.000881,0.000529,0.000000,0.000000,0.002643,0.001762,2.400000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483424,522913,36305,2,1,0.001821,0.000000,0.000000,0,0.0,0.000000,0.000000,1.200000,0.136461,6.200000,143.110500,0,0.002732,0.005464,0.000000,0.000911,0.000546,0.000000,0.000000,0.002732,0.001821,3537.918919,2
486991,145840,88979,3,1,0.053743,3.000000,3.333333,2,0.0,0.000000,0.100000,2.100000,0.341657,8.600000,83.051415,7,0.007678,0.013436,0.006718,0.000480,0.001727,0.000000,0.000000,0.009597,0.000000,389.769231,2
487280,428367,38815,2,1,0.006410,1.500000,1.000000,3,0.0,0.000000,0.000000,0.193548,0.176521,7.894118,148.583825,0,0.004579,0.051282,0.000000,0.005037,0.013553,0.024725,0.001832,0.047619,0.003663,157.071429,2
487736,105750,74545,2,1,0.005736,0.000000,0.000000,0,0.0,0.000000,0.000000,0.234043,0.530261,17.093750,76.225133,0,0.002868,0.041109,0.000000,0.003824,0.004589,0.005736,0.003824,0.012428,0.008604,1.000000,2


In [ ]:
train5=train4[(train4.userId>min_threshold5)]
train5.shape

(486445, 27)

In [ ]:
train5.describe()

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
count,486445.000000,4.864450e+05,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000,486445.000000
mean,271785.095041,4.245096e+07,1.976252,1.213171,0.044686,0.343795,0.399055,0.684955,0.000272,0.000979,0.012602,0.176836,0.341456,14.836067,82.957566,0.300825,0.016496,0.389174,0.001890,0.073805,0.066424,0.032507,0.140705,0.149303,0.157217,142.561861,1.737808
std,156604.060006,1.814428e+07,0.429642,0.409548,0.286555,0.812537,0.921492,2.276984,0.011956,0.020398,0.154009,0.645566,0.195412,39.764625,76.766249,7.278103,0.176285,0.890659,0.025309,0.207049,0.169895,0.189676,0.395482,0.442313,0.493473,708.049014,1.086923
min,0.000000,9.148800e+04,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000653,1.000000,0.233000,0.000000,0.000000,0.000972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,136386.000000,3.541434e+07,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199478,5.640000,30.623392,0.000000,0.000000,0.040816,0.000000,0.000000,0.002256,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,271810.000000,4.341069e+07,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329674,8.500000,62.316235,0.000000,0.000000,0.125000,0.000000,0.003968,0.015873,0.000000,0.011725,0.014493,0.015873,0.000000,1.000000
75%,407224.000000,5.373984e+07,2.000000,1.000000,0.008547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147826,0.460442,13.000000,111.938044,0.000000,0.009091,0.363636,0.000000,0.059829,0.058824,0.000000,0.116667,0.118110,0.121951,178.428571,2.000000
max,543196.000000,7.904203e+07,3.000000,2.000000,9.184874,4.000000,4.000000,35.000000,2.333333,3.000000,27.333333,262.666667,1.000000,1127.250000,7541.026000,3228.000000,63.388889,39.991935,4.516279,13.110294,10.738462,19.612903,27.905983,31.675214,55.153846,17306.625000,4.000000


In [ ]:
min_threshold6,max_threshold6=train['Unnamed: 0'].quantile([0.001,0.999])
print(min_threshold6,max_threshold6)

555.7520000000001 542657.1240000001


In [ ]:
train5[train5['Unnamed: 0']<min_threshold6]

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
1244,77,1741176,1,2,0.001799,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.357270,104.962963,190.642286,0,0.0,0.012590,0.000000,0.000000,0.009712,0.000000,0.000000,0.043165,0.005396,0.000000,1
2214,174,2155181,1,1,0.011215,0.0,0.0,0,0.0,0.0,0.0,0.090909,0.096403,19.000000,243.793250,2,0.0,0.014953,0.003738,0.000000,0.007103,0.000000,0.000000,0.000000,0.035514,0.000000,1
3995,1,985371,1,1,0.001603,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.049384,5.555556,205.073500,0,0.0,0.022436,0.000000,0.007212,0.000000,0.000000,0.014423,0.000000,0.000000,0.000000,1
4677,362,6461972,1,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.033821,4.571429,132.127333,0,0.0,0.017995,0.000000,0.000000,0.003599,0.000000,0.000000,0.000000,0.017995,0.000000,1
5265,308,2834349,1,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.361639,54.000000,149.320000,0,0.0,0.002155,0.000000,0.000000,0.000431,0.000000,0.002155,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485670,103,1827201,1,1,0.001818,0.0,0.0,0,0.0,0.0,0.0,0.019231,0.273081,14.481481,105.147444,0,0.0,0.092727,0.000000,0.042727,0.002545,0.000000,0.000000,0.098182,0.000000,0.000000,1
485799,25,1516912,1,1,0.001745,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.667700,78.000000,116.819000,0,0.0,0.001745,0.000000,0.000873,0.000000,0.000000,0.000000,0.000000,0.001745,0.000000,1
486654,178,2174086,1,1,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.007738,2.000000,258.455000,0,0.0,0.001876,0.000000,0.000000,0.000375,0.000000,0.000000,0.001876,0.000000,0.000000,1
486790,180,2196790,1,1,0.007547,2.0,2.0,0,0.0,0.0,0.0,0.300000,0.145855,17.666667,156.978833,1,0.0,0.015094,0.001887,0.011321,0.000000,0.009434,0.013208,0.000000,0.000000,111.666667,1


In [ ]:
train6=train5[(train5['Unnamed: 0']>min_threshold6)]
train6.shape

(485956, 27)

In [ ]:
y = train['age_group']
X=train.drop(['age_group'], axis =1)

In [ ]:
sqrt_col=['avgComments',
 'creations',
 'content_views',
 'num_of_comments',
 'weekends_trails_watched_per_day',
 'weekdays_trails_watched_per_day','following_rate','avgComments','slot2_trails_watched_per_day','slot1_trails_watched_per_day', 'avgt2','slot4_trails_watched_per_day','slot3_trails_watched_per_day','followers_avg_age','following_avg_age','max_repetitive_punc','num_of_hashtags_per_action','avgCompletion','emoji_count_per_action','punctuations_per_action','number_of_words_per_action','avgTimeSpent']
log_col=['avgDuration']

In [ ]:
for col in sqrt_col:
    X[col] = np.sqrt(X[col])
    test[col]=np.sqrt(test[col])

for col in log_col:
    X[col]=(X[col]-X[col].min()+1).transform(np.log)
    test[col]=(test[col]-test[col].min()+1).transform(np.log)

In [ ]:
%matplotlib inline

# Start Python Imports
import math, time, random, datetime
from random import shuffle

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize, MinMaxScaler

# Machine learning
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import xgboost
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.svm import SVR




In [ ]:
#apply SelectKBest class to extract top 10 best features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
bestfeatures = SelectKBest(score_func=chi2, k=26)
fit = bestfeatures.fit(X,y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

In [ ]:
print(featureScores.nlargest(26,'Score')) 

                              Specs         Score
1                            userId  3.401960e+11
0                        Unnamed: 0  1.777622e+09
25                            avgt2  2.209679e+06
7               max_repetitive_punc  8.416338e+04
11       number_of_words_per_action  6.641820e+04
6                 following_avg_age  6.074967e+04
5                 followers_avg_age  5.692391e+04
15                      avgComments  3.213721e+04
16                        creations  2.408705e+04
4                    following_rate  2.076759e+04
24     slot4_trails_watched_per_day  1.256428e+04
13                     avgTimeSpent  1.250221e+04
23     slot3_trails_watched_per_day  1.007599e+04
17                    content_views  9.858925e+03
19  weekends_trails_watched_per_day  7.211991e+03
22     slot2_trails_watched_per_day  6.721183e+03
21     slot1_trails_watched_per_day  6.685476e+03
20  weekdays_trails_watched_per_day  6.287462e+03
10          punctuations_per_action  5.543064e+03


In [ ]:
#apply SelectKBest class to extract top 10 best features
bestfeatures_f = SelectKBest(score_func=f_classif, k=39)
fit_f = bestfeatures.fit(X,y)

In [ ]:
dfscores_f = pd.DataFrame(fit_f.scores_)
dfcolumns_f = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores_f = pd.concat([dfcolumns_f,dfscores_f],axis=1)
featureScores_f.columns = ['Specs','Score']  #naming the dataframe columns

In [ ]:
print(featureScores_f.nlargest(39,'Score')) 

                              Specs         Score
1                            userId  3.401960e+11
0                        Unnamed: 0  1.777622e+09
25                            avgt2  2.209679e+06
7               max_repetitive_punc  8.416338e+04
11       number_of_words_per_action  6.641820e+04
6                 following_avg_age  6.074967e+04
5                 followers_avg_age  5.692391e+04
15                      avgComments  3.213721e+04
16                        creations  2.408705e+04
4                    following_rate  2.076759e+04
24     slot4_trails_watched_per_day  1.256428e+04
13                     avgTimeSpent  1.250221e+04
23     slot3_trails_watched_per_day  1.007599e+04
17                    content_views  9.858925e+03
19  weekends_trails_watched_per_day  7.211991e+03
22     slot2_trails_watched_per_day  6.721183e+03
21     slot1_trails_watched_per_day  6.685476e+03
20  weekdays_trails_watched_per_day  6.287462e+03
10          punctuations_per_action  5.543064e+03


In [ ]:
X_train = X.drop(['avgCompletion','num_of_hashtags_per_action','gender'], axis=1)
X_test = test.drop(['avgCompletion','num_of_hashtags_per_action','gender'], axis=1)

In [ ]:
xbg_classifier = xgboost.XGBClassifier( random_state = 51)

booster=['gbtree']    

## Hyper Parameter Optimization

hyperparameter_grid = {
        'n_estimators' : [100, 500, 900, 1100, 1500],      # no. of decision trees used
        'min_child_weight': [1, 2, 3, 5, 7, 9],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120],
        'learning_rate' : [0.005,0.01,0.03, 0.05, 0.15,0.3, 0.45, 0.55]
        }

# Set up the random search with 4-fold cross validation
folds = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 51)

random_cv = RandomizedSearchCV(estimator=xbg_classifier,
            param_distributions=hyperparameter_grid,
            cv=skf.split(X,y),
            scoring = 'f1_micro',n_jobs = 4,
            verbose = 5, 
            return_train_score = True)

In [ ]:
random_cv.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
pip install catboost

     |████████████████████████████████| 66.3MB 54kB/s 


In [ ]:
import catboost
from catboost import CatBoostClassifier, Pool, CatBoostRegressor,cv

In [ ]:
indices_cat = []
for col in X.columns:
        indices_cat.append(X.columns.get_loc(col))

indices_cat.sort()
indices_cat

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [ ]:

model_cat = CatBoostClassifier(random_state = 51,eval_metric = 'f1_micro')

random_grid_cat = {'learning_rate': [0.05, 0.08, 0.1, 0.15, 0.2, 0.3],
        'depth': [4, 6,10,15,20,30,40,50,60,70],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

randomized_search_cat = model_cat.randomized_search(random_grid_cat, train_pool_cat, cv=5,plot=True)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=11)
neigh.fit(X_train,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
y_test_pred=neigh.predict(X_test)
output = pd.DataFrame({'prediction': y_test_pred})
output.to_csv('submission92.csv',index=False)

In [ ]:
rfc=RandomForestClassifier(n_estimators=500,n_jobs=-1,random_state=7,min_samples_leaf=3)
rfc.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=7, verbose=0,
                       warm_start=False)

In [ ]:
dt=DecisionTreeClassifier(random_state=7,min_samples_leaf=3)
dt.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=7, splitter='best')

In [ ]:
y_test_pred1= rfc.predict_proba(test)
y_test_pred2=neigh.predict_proba(test)
y_test_pred3=dt.predict_proba(test)
y_test_pred=(y_test_pred1*0+y_test_pred2*0+y_test_pred3*1)

y_test_pred=np.argmax(y_test_pred,axis=1)+1
y_test_pred

array([1, 1, 1, ..., 4, 1, 1])

In [ ]:
y_test_pred=dt.predict(test)

In [ ]:
y_test_pred.shape

(54320,)

In [ ]:
print(cross_val_score(rfc, X, y, cv=9,scoring='f1_micro').mean())
print(cross_val_score(neigh, X, y, cv=9,scoring='f1_micro').mean())

In [ ]:
output = pd.DataFrame({'prediction': y_test_pred})
output.to_csv('submission80.csv',index=False)